# Bagging

### Data Loading

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# 현재경로 확인
os.getcwd()

In [2]:
data = pd.read_csv('../00_Data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [4]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


### 의미없는 변수 제거

In [5]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) 

### 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

### 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행 
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


### 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models)

In [9]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant='add') # Bias 추가
sm_model = sm.OLS(train_y, sm_train_x) # 모델 구축
fitted_sm_model = sm_model.fit() # 학습 진행
fitted_sm_model.summary() # 학습 모델 구조 확인

C:\Users\Hongjin\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Thu, 24 Nov 2022   Prob (F-statistic):               0.00
Time:                        20:17:59   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
# 결과 확인
sm_test_x = sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)

print("RMSE: {}".format(sqrt(mean_squared_error(sm_model_predict, test_y)))) # RMSE
print(fitted_sm_model.params) # 회귀계수

RMSE: 239804.29670858165
const           7.185671e+06
bathrooms       1.302689e+05
bedrooms       -2.224791e+03
condition       1.641020e+04
floors          1.946305e+03
grade           1.955909e+05
waterfront      7.555423e+05
yr_built       -4.300787e+03
yr_renovated    1.273246e+01
dtype: float64


C:\Users\Hongjin\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


### Bagging 한 결과가 일반적인 결과보다 좋은지 확인

In [22]:
# Bagging
import random

bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스 뽑기
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, train_x.shape[0]만큼 뽑음
    print(len(set(random_data_index)))
    
    sm_train_x = train_x.iloc[random_data_index, :] # 랜텀으로 뽑은 인덱스에 해당되는 데이터 추출
    sm_train_y = train_y.iloc[random_data_index] # 랜텀으로 뽑은 인덱스에 해당되는 데이터 추출
    
    # train
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    
    # test
    sm_test_x = sm.add_constant(test_x, has_constant='add')
    sm_model_predict = fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(sm_model_predict) # test 결과값들 하나씩 담기

9537
9607
9570
9571
9530
9562
9546
9449
9624
9562


In [23]:
bagging_predict_result[0] # 0 ~ 9, 10번의 예측을 하였기 때문에 10개의 결과가 생성

735      5.626462e+05
2830     7.230131e+05
4106     1.120959e+06
16218    1.483390e+06
19964    6.994261e+05
             ...     
12606    6.019952e+05
14393    6.807654e+05
6899     3.230883e+05
85       9.028413e+05
21363    4.336199e+05
Length: 6484, dtype: float64

In [25]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균 계산
bagging_predict = []
for idx_test in range(test_x.shape[0]): # 테스트 데이터 개수만큼 반복
    temp_predict = []
    for idx in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[idx].values[idx_test]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [26]:
bagging_predict

[563475.7572315828,
 711081.4395975156,
 1119226.1188219632,
 1475091.8253160068,
 699712.1212354845,
 386332.63567076385,
 787036.88410428,
 483903.9597152953,
 499853.49867190386,
 535711.1217348882,
 641420.4810364314,
 404031.1710803168,
 267829.9797356345,
 280555.20438622293,
 335485.5528236343,
 1263315.6358896475,
 323740.92230699066,
 1035065.2393867348,
 261682.79032518022,
 603803.1548455664,
 392190.4262409425,
 1302194.9793338487,
 821459.7750459814,
 581781.5218339489,
 596538.007121354,
 569933.3112771504,
 265635.2582770829,
 40143.44856489909,
 564666.918375422,
 637855.6430015498,
 563700.5986384143,
 457976.5748538788,
 553652.7241160807,
 678841.8765580251,
 409210.9251837148,
 875452.6204056408,
 939325.8170140728,
 637548.734804481,
 390962.41460545384,
 1081022.1938351856,
 453030.4121977195,
 148287.86534258086,
 490681.35392037744,
 221599.5752528833,
 65776.63571350313,
 -44562.50252273015,
 248588.72680155403,
 284391.5999300407,
 362997.0777093204,
 717478.9